# Exercises

Data Acquisition     

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.     

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.     

1. Read the case, department, and source data into their own spark dataframes.     
1. Let's see how writing to the local disk works in spark:
    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?
1. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [2]:
# 1. Read the case, department, and source data into their own spark dataframes
srcdf = spark.read.csv("data/source.csv", sep=",", header=True, inferSchema=True)
deptdf = spark.read.csv("data/dept.csv", sep=",", header=True, inferSchema=True)
casedf = spark.read.csv("data/case.csv", sep=",", header=True, inferSchema=True)

In [3]:
# to specify schema instead of infering
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
    ]
)

spark.read.csv("data/source.csv", header=True, schema=schema)

DataFrame[source_id: string, source_username: string]

In [4]:
# 2.a Write the code necessary to store the source data in both csv and json format,
# store these as sources_csv and sources_json
srcdf.write.csv("data/sources.csv", mode="overwrite")
srcdf.write.json("data/sources.json", mode="overwrite")

In [5]:
# 2.b Inspect your folder structure. What do you notice?
# creates folder with success message and file with complicated name

In [6]:
# 3.a Inspect the data in your dataframes. Are the data types appropriate?
srcdf.show()
srcdf.printSchema
# source id and username are both strings, these are both good as ID is an identifier, not mathmatical number

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



<bound method DataFrame.printSchema of DataFrame[source_id: string, source_username: string]>

In [7]:
deptdf.show()
deptdf.printSchema
# all are strings, looks correct

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

<bound method DataFrame.printSchema of DataFrame[dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string]>

In [8]:
casedf.show(3, vertical=True)
casedf.printSchema
# need to change dates to timestamp, YES/NO to booleen, council to string, etc.

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

<bound method DataFrame.printSchema of DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: int]>

In [9]:
# FROM LESSON do this:
# Renamed SLA_due_date
# Converted to booleans
# Padded council_district with 0s
# Handled dates
# Normalized the text in the address
# Extracted zipcode from address
# Created a case lifetime feature
# Joined department data
# 3.b Write the code necessary to cast the values to the appropriate types
casedf = casedf.withColumnRenamed("SLA_due_date", "case_due_date")                          

In [10]:

casedf.groupby('case_late', 'case_closed').count().show()

+---------+-----------+------+
|case_late|case_closed| count|
+---------+-----------+------+
|       NO|        YES|735616|
|      YES|        YES| 87978|
|       NO|         NO| 11585|
|      YES|         NO|  6525|
+---------+-----------+------+



In [11]:
# note the quotation syntax in expr
casedf = casedf.withColumn('case_late', expr("case_late == 'YES'"))

In [12]:
casedf = casedf.withColumn('case_closed', expr("case_closed == 'YES'"))

In [13]:
casedf.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | false                
 num_days_late        | -2.0126041

In [14]:
# df = df.withColumn("council_district", format_string("%03d", col("council_district")))
casedf = casedf.withColumn('council_district', format_string("%03d", col("council_district")))

In [15]:
casedf.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | false                
 num_days_late        | -2.0126041

In [16]:
print("--- Before handling dates")
casedf.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
casedf = (
    casedf.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

print("--- After")
casedf.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- Before handling dates
+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:

In [17]:
# casedf = casedf.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
# casedf = casedf.withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
# casedf = casedf.withColumn("case_due_date", to_timestamp("case_due_date", fmt))

In [18]:
casedf.printSchema()
casedf.show(3, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true      

In [19]:
# lowercase address and remove whitespace
casedf = casedf.withColumn("request_address", lower(trim("request_address")))

In [20]:
# use regex to capture and extract zip code into new column
casedf = casedf.withColumn("zipcode", regexp_extract("request_address", r"(\d+)$", 1))

In [21]:
# create case age, days to close, and case lifetime columns
casedf = casedf.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))
casedf = casedf.withColumn("days_to_close", datediff("case_closed_date", "case_opened_date"))
casedf = casedf.withColumn("case_lifetime", when(col("case_closed"), col("days_to_close")).otherwise(col("case_age")))

In [22]:
# show records where case is open (not closed)
casedf.where(~ col("case_closed")).show(vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014128388           
 case_opened_date     | 2018-01-02 09:39:00  
 case_closed_date     | null                 
 case_due_date        | 2018-01-09 09:39:00  
 case_late            | true                 
 num_days_late        | 211.5974884          
 case_closed          | false                
 dept_division        | 311 Call Center      
 service_request_type | Complaint            
 SLA_days             | 7.0                  
 case_status          | Open                 
 source_id            | mt13131              
 request_address      | 7326  westglade p... 
 council_district     | 006                  
 zipcode              | 78227                
 case_age             | 1064                 
 days_to_close        | null                 
 case_lifetime        | 1064                 
-RECORD 1------------------------------------
 case_id              | 1014128790           
 case_opened_date     | 2018-01-02

In [23]:
# join dept table
casedf = (
    casedf
    # left join on dept_division
    .join(deptdf, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(deptdf.dept_division)
    .drop(deptdf.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

casedf.show(2, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_close        | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
-RECORD 1-------------------------

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [24]:
# "days past SLA" means days past SLA due date? which we renamed case_due_date?
casedf = casedf.withColumn("days_past_SLAdue", datediff(current_timestamp(), "case_due_date"))

In [25]:
# so for open cases that is the difference between the case due date and the current date?
# then the second part is the date difference between the open date and current date for open cases? which is the case age?

In [26]:
casedf.where(~ col("case_closed")).show(vertical=True)

-RECORD 0------------------------------------
 dept_division        | 311 Call Center      
 case_id              | 1014128388           
 case_opened_date     | 2018-01-02 09:39:00  
 case_closed_date     | null                 
 case_due_date        | 2018-01-09 09:39:00  
 case_late            | true                 
 num_days_late        | 211.5974884          
 case_closed          | false                
 service_request_type | Complaint            
 SLA_days             | 7.0                  
 case_status          | Open                 
 source_id            | mt13131              
 request_address      | 7326  westglade p... 
 council_district     | 006                  
 zipcode              | 78227                
 case_age             | 1064                 
 days_to_close        | null                 
 case_lifetime        | 1064                 
 department           | Customer Service     
 dept_subject_to_SLA  | true                 
 days_past_SLAdue     | 1057      

2. How many Stray Animal cases are there?

In [27]:
(
    casedf.filter(col("service_request_type") == lit("Stray Animal"))
    .groupBy("case_closed", "service_request_type")
    .count()
    .sort(col("count").desc())
    .show()
)

+-----------+--------------------+-----+
|case_closed|service_request_type|count|
+-----------+--------------------+-----+
|       true|        Stray Animal|26745|
|      false|        Stray Animal|   15|
+-----------+--------------------+-----+



3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?


In [28]:
(
    casedf.filter(col("service_request_type") != lit("Officer Standby"))
    .count()
)

838691

4. Convert the council_district column to a string column.

In [29]:
casedf.printSchema

<bound method DataFrame.printSchema of DataFrame[dept_division: string, case_id: int, case_opened_date: timestamp, case_closed_date: timestamp, case_due_date: timestamp, case_late: boolean, num_days_late: double, case_closed: boolean, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: string, zipcode: string, case_age: int, days_to_close: int, case_lifetime: int, department: string, dept_subject_to_SLA: boolean, days_past_SLAdue: int]>

5. Extract the year from the case_closed_date column.

In [30]:
casedf = casedf.withColumn("year", year("case_closed_date"))

In [31]:
casedf.show(2, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_close        | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 days_past_SLAdue     | 66        

6. Convert num_days_late from days to hours in new columns num_hours_late.

In [32]:
casedf = casedf.withColumn("num_hours_late", (col("num_days_late") * 24))

In [33]:
casedf.show(2, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_close        | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 days_past_SLAdue     | 66        

7. Join the case data with the source and department data.

In [34]:
srcdf.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [35]:
casedf.show(2, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_close        | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 days_past_SLAdue     | 66        

In [36]:
# join source table, dept table already joined
casedf = casedf.join(srcdf, "source_id", "left")

In [37]:
casedf.show(2, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_close        | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 days_past_SLAdue     | 66        

8. Are there any cases that do not have a request source?

In [38]:
casedf.where(casedf.source_username == None).show(vertical=True)

(0 rows)



9. What are the top 10 service request types in terms of number of requests?

In [39]:
(
    casedf
    .groupBy("service_request_type")
    .count()
    .sort(col("count").desc())
    .show(10)
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



10. What are the top 10 service request types in terms of average days late?

In [40]:
(
    casedf
    .groupBy("service_request_type")
    .mean("num_days_late")
    .sort(col("avg(num_days_late)").desc())
    .show(10)
)

+--------------------+------------------+
|service_request_type|avg(num_days_late)|
+--------------------+------------------+
|  Zoning: Junk Yards|175.95636210420943|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...|153.99724039428568|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|142.11255641500003|
|Zoning: Recycle Yard|135.92851612479797|
|Donation Containe...|131.75610506358706|
|License Requied U...|128.79828704142858|
|Traffic Signal Gr...|101.79846062200002|
|           Complaint|  72.8705023031169|
+--------------------+------------------+
only showing top 10 rows



11. Does number of days late depend on department?

In [41]:
(
    casedf
    .groupBy("department")
    .mean("num_days_late")
    .sort(col("avg(num_days_late)").desc())
    .show()
)

+--------------------+-------------------+
|          department| avg(num_days_late)|
+--------------------+-------------------+
|    Customer Service|  59.73709149630077|
|         Solid Waste| -2.200057513672164|
|        Metro Health| -4.911766979607004|
|  Parks & Recreation| -5.251521960055145|
|Trans & Cap Impro...|-20.612837354052708|
|DSD/Code Enforcement| -38.36938892614506|
|Animal Care Services|-226.51783940550334|
|        City Council|               null|
+--------------------+-------------------+



12. How do number of days late depend on department and request type?

In [42]:
(
    casedf
    .groupBy("department", "service_request_type")
    .mean("num_days_late")
    .sort(col("avg(num_days_late)").desc())
    .show()
)



+--------------------+--------------------+------------------+
|          department|service_request_type|avg(num_days_late)|
+--------------------+--------------------+------------------+
|DSD/Code Enforcement|  Zoning: Junk Yards|175.95636210420943|
|DSD/Code Enforcement|Labeling for Used...|162.43032902285717|
|DSD/Code Enforcement|Record Keeping of...|153.99724039428568|
|DSD/Code Enforcement|Signage Requied f...|151.63868055333333|
|DSD/Code Enforcement|Storage of Used M...|142.11255641500003|
|DSD/Code Enforcement|Zoning: Recycle Yard|135.92851612479797|
|DSD/Code Enforcement|Donation Containe...|131.75610506358706|
|DSD/Code Enforcement|License Requied U...|128.79828704142858|
|Trans & Cap Impro...|Traffic Signal Gr...|101.79846062200002|
|    Customer Service|           Complaint|  72.8705023031169|
|DSD/Code Enforcement|             Vendors| 66.54809898507798|
|  Parks & Recreation|Reservation Assis...|       66.03116319|
|DSD/Code Enforcement|   No Address Posted| 59.87564073

**Per Zack this is BONUS**

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [43]:
casedf.show(2, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_close        | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 days_past_SLAdue     | 66        

In [44]:
# find the max date in the dataset
casedf.groupby("department").agg(max("case_opened_date"), max("case_closed_date"), max("case_due_date")).show()

+--------------------+---------------------+---------------------+-------------------+
|          department|max(case_opened_date)|max(case_closed_date)| max(case_due_date)|
+--------------------+---------------------+---------------------+-------------------+
|         Solid Waste|  2018-08-08 10:38:00|  2018-08-08 10:38:00|2018-08-22 10:19:00|
|Animal Care Services|  2018-08-08 10:37:00|  2018-08-08 10:35:00|2021-05-03 10:20:00|
|Trans & Cap Impro...|  2018-08-08 10:36:00|  2018-08-08 10:38:00|2022-05-27 15:07:00|
|  Parks & Recreation|  2018-08-08 10:31:00|  2018-08-08 10:16:00|2019-08-06 08:00:00|
|    Customer Service|  2018-08-07 18:14:00|  2018-08-08 10:06:00|2018-08-17 08:30:00|
|        Metro Health|  2018-08-08 10:25:00|  2018-08-07 15:35:00|2018-08-22 10:25:00|
|        City Council|  2018-08-08 10:33:00|  2018-07-12 13:06:00|               null|
|DSD/Code Enforcement|  2018-08-08 10:38:00|  2018-08-08 10:37:00|2018-12-13 10:34:00|
+--------------------+---------------------

In [45]:
casedf.select(
    regexp_extract("trans", r"^(\w+)\(", 1).alias("regexp_extract"),
    regexp_replace("trans", r"\(.+$", "").alias("regexp_replace"),
    when(mpg.trans.like("auto%"), "auto")
    .otherwise("manual")
    .alias("when + like"),
).show()

NameError: name 'mpg' is not defined